# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [54]:
#Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#numpy and matpotlib imports
import numpy as np
import matplotlib.pyplot as plt

#Torch imports
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

#Torchvision imports
from torchvision import datasets,transforms, models
from torchvision.datasets import ImageFolder
from torchvision.models import vgg16
import torchvision as torchvision

#PIL imports
from PIL import Image

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [55]:
#data information
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [56]:
#TODO: Define your transforms for the training, validation, and testing sets

#data_transforms with Resizing
data_transforms = transforms.Compose([transforms.RandomRotation(30), #randomly rotates image
                                      transforms.RandomResizedCrop(224), # resizes to 224 pixels
                                      transforms.RandomHorizontalFlip(), # random horizontal flip
                                      transforms.ToTensor(), # creates a tensor
                                      transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)) # normalizes colors
                                     ])

test_transforms = transforms.Compose([transforms.RandomResizedCrop(224), # resizes to 224 pixels
                                      transforms.RandomHorizontalFlip(30), # random horizontal flip
                                      transforms.ToTensor(), # creates a tensor
                                      transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)) # normalizes colors
                                     ])
valid_transforms = transforms.Compose([transforms.RandomResizedCrop(224), # resizes to 224 pixels
                                      transforms.RandomHorizontalFlip(30), # random horizontal flip
                                      transforms.ToTensor(), # creates a tensor
                                      transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)) # normalizes colors
                                     ])
# TODO: Load the datasets with ImageFolder
image_datasets = ImageFolder(train_dir, transform = data_transforms) #loads the training set
image_test_dataset = ImageFolder(test_dir, transform = test_transforms) #loads the test set
image_valid_dataset = ImageFolder(valid_dir, transform = valid_transforms) #loads the validation set

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = torch.utils.data.DataLoader(image_datasets, batch_size = 64 ,shuffle = True) #shuffling on the training set
testloader = torch.utils.data.DataLoader(image_test_dataset, batch_size = 64) # no shuffling on the training set
validloader = torch.utils.data.DataLoader(image_valid_dataset, batch_size = 64) # no shuffling on the training set



In [13]:
dataiter = iter(dataloaders) 
images, labels = dataiter.next()
print(images.shape,labels.shape) #shape test

torch.Size([64, 3, 224, 224]) torch.Size([64])


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [14]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)


# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

<font color='red'>**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.</font>

In [15]:
# TODO: Build and train your network

'''
Instructions to keep in mind:

DONE - Load a pre-trained network (If you need a starting point,the VGG networks work great and are straightforward to use) 
  
DONE - Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout 

DONE - Train the classifier layers using backpropagation using the pre-trained network to get the features. 

DONE - Track the loss and accuracy on the validation set to determine the best hyperparameters

'''

'\nInstructions to keep in mind:\n\nDONE - Load a pre-trained network (If you need a starting point,the VGG networks work great and are straightforward to use) \n  \nDONE - Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout \n\nDONE - Train the classifier layers using backpropagation using the pre-trained network to get the features. \n\nDONE - Track the loss and accuracy on the validation set to determine the best hyperparameters\n\n'

In [16]:
#use pre-trained model vgg16
pre_trained_model = models.vgg16(pretrained = True)#pretrain set to boolean value of True
pre_trained_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [17]:
#Freeze parameters
for parameter in pre_trained_model.parameters():
    parameter.requires_grad = False

#edit vgg16 classifier

#beginning of classifier
classifier = nn.Sequential(
    (nn.Linear(25088,512)), #first layer 
    (nn.Dropout(0.2)), # Dropout function call
    (nn.ReLU()), # call to ReLU
    (nn.Linear(512,102)), # second layer
    (nn.ReLU()), # call to ReLU
    (nn.LogSoftmax(dim =1)) # LogSoftmax function call from torch.nn
)

pre_trained_model.classifier = classifier #assigns pre_trained_model classifier 

loss_criterion = nn.CrossEntropyLoss() #creates a loss_criterion

optimizer = optim.Adam(pre_trained_model.classifier.parameters(), lr = 0.01) # create the optimizer

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

pre_trained_model.to(device )

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [19]:
epochs = 5
running_loss = 0 #loss from running through training set
training_loss = 0

for e in range(epochs):
    
    
    #beginning of for loop
    for images,labels in dataloaders:
        
        images, labels = images.to(device ), labels.to(device)
        optimizer.zero_grad()
        
        train_forward = pre_trained_model.forward(images) # moves forward through the network to train the model
        
        loss = loss_criterion(train_forward, labels) # loss calculator
        
        loss.backward() # goes backword through the network
        
        optimizer.step() # steps through optimizer 

        running_loss += loss.item() #checks loss
        
        pre_trained_model.train() #train the model
        
        print('ran thru', e)
     # beginning of else statement
    else:#referenced from Udacity's transfer learning unit
        test_loss = 0
        test_accuracy = 0
        print('starting')
        with torch.no_grad():#takes out gradient
            pre_trained_model.eval() #evaluates model
            
            # beginning of for loop
            for images, labels in dataloaders:
                images, labels = images.to(device), labels.to(device)
                train_forward = pre_trained_model.forward(images) # restates trainforward with no gradient
                test_loss += loss_criterion(train_forward, labels) # restates testloss with no gradient
                
                class_prob = torch.exp(train_forward) #calculates probability
                top_p, top_class = class_prob.topk(1, dim=1) # 
                
                equals = top_class == labels.view(*top_class.shape) 
                test_accuracy += torch.mean(equals.type(torch.FloatTensor))
            print("VALIDATION TEST:\n",
            "Training loss: {:.3f}..".format(training_loss/len(dataloaders)),
            "Test Loss: {:.3f}.. ".format(test_loss/len(dataloaders)),
            "Accuracy: {:.3f}".format(test_accuracy/len(dataloaders)))
            #end of for loop   
            running_loss = 0
            training_loss = 0
    # end of else statement
                
        pre_trained_model.train() #trains model 

        
        

ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0
ran thru 0

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set
epochs = 10
running_loss = 0 #loss from running through training set
training_loss = 0

for e in range(epochs):
    
    
    #beginning of for loop
    for images,labels in validloader:
        
        images, labels = images.to(device ), labels.to(device)
        optimizer.zero_grad()
        
        train_forward = pre_trained_model.forward(images) # moves forward through the network to train the model
        
        loss = loss_criterion(train_forward, labels) # loss calculator
        
        loss.backward() # goes backword through the network
        
        optimizer.step() # steps through optimizer 

        running_loss += loss.item() #checks loss
        
        pre_trained_model.train() #train the model
        
     # beginning of else statement
    else:#referenced from Udacity's transfer learning unit
        test_loss = 0
        test_accuracy = 0

        with torch.no_grad():#takes out gradient
            pre_trained_model.eval() #evaluates model
            
            # beginning of for loop
            for images, labels in validloader:
                images, labels = images.to(device), labels.to(device)
                train_forward = pre_trained_model.forward(images) # restates trainforward with no gradient
                test_loss += loss_criterion(train_forward, labels) # restates testloss with no gradient
                
                class_prob = torch.exp(train_forward) #calculates probability
                top_p, top_class = class_prob.topk(1, dim=1) # 
                
                equals = top_class == labels.view(*top_class.shape) 
                test_accuracy += torch.mean(equals.type(torch.FloatTensor))
            print("VALIDATION TEST:\n",
            "Training loss: {:.3f}..".format(training_loss/len(dataloaders)),
            "Test Loss: {:.3f}.. ".format(test_loss/len(dataloaders)),
            "Accuracy: {:.3f}".format(test_accuracy/len(dataloaders)))
            #end of for loop   
            running_loss = 0
            training_loss = 0
    # end of else statement
                


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 

pre_trained_model.class_to_idx = dataloaders.dataset.class_to_idx

checkpoint = {'input_size': 25088,
              'output_size': 102,
              'dropout': 0.2,
              'state_dict': pre_trained_model.state_dict(),
              'classifer': classifier,
              'class_to_idx': pre_trained_model.class_to_idx,
              'epoch': epochs,
              'optimizer': optimizer.state_dict()}

#save checkpoint:
torch.save(optimizer.state_dict(),'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = pre_trained_model(checkpoint['input_size'],
                              checkpoint['output_size'],
                              checkpoint['state_dict'],
                              checkpoint['classifer'],
                              checkpoint['class_to_idx'],
                              checkpoint['epoch'],
                                )
    pre_trained_model.load_state_dict(checkpoint['state_dict'])
    
    return model

load_checkpoint('checkpoint.pth')

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [79]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    std = [0,485,0.456,0,406]
    mean = [0.485,0.456,0.406]
    div = 0
    # TODO: Process a PIL image for use in a PyTorch model
    for i in pre_trained_model.class_to_idx:
        with Image.open(i) as I:
            I.resize(256)
            I.CenterCrop(224),
            np_image = np.array(i)
            for val in np_image:
                for valu in means:
                    subt +=val
                    for value in std:
                        div += subt/value

            np_image.append(div)
            ndarray.tranpose(2,0)

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [80]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [81]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    # TODO: Implement the code to predict the class from an image file
    prob = torch.exp(model(image_path)) #creates probability
    top_p, top_q = prob.top_k(topk, dim = 1) # looks for first top_k 
    
    plt.x = image_path
    
    plt.y = top_q
    
    plt.bar(x,y)
    
    plt.xlabel('names of flowers')
    plt.ylabel('probability')
    plt.show()
    
    return top_p, top_q

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [72]:
# TODO: Display an image along with the top 5 classes

In [73]:
img, img_label = predict(cat_to_name, pre_trained_model, topk)

plt.imshow(img)

predict(cat_to_name, pre_trained_model, topk)


FileNotFoundError: [Errno 2] No such file or directory: '1'

<font color='red'>**Reminder for Workspace users:** If your network becomes very large when saved as a checkpoint, there might be issues with saving backups in your workspace. You should reduce the size of your hidden layers and train again. 
    
We strongly encourage you to delete these large interim files and directories before navigating to another page or closing the browser tab.</font>

In [ ]:
# TODO remove .pth files or move it to a temporary `~/opt` directory in this Workspace